# Routing: Semantic

![semantic routing](../images/images-semantic.png)

**Semantic Routing** is a technique to direct queries to the most suitable data source or system based on the **meaning** or **context** of the query rather than relying on predefined rules or exact keywords. It leverages semantic understanding, often using embeddings or machine learning models, to analyze the intent behind a query and make intelligent routing decisions.

### The Problem Semantic Routing Solves
Sometimes, queries don’t fit neatly into predefined rules or contain exact keywords. They may:
- Be phrased in unexpected ways.
- Require interpretation of the context to understand their true intent.

**Semantic Routing** solves this by focusing on the query's meaning rather than its surface structure, ensuring accurate routing even for ambiguous or unconventional queries.


### How Semantic Routing Works
1. **Understand the Query**:
   - The system processes the query using a semantic analysis model (e.g., an embedding model like Sentence Transformers) to understand its meaning.
   - Example:
     - Query: "How can I keep my web server safe from intrusions?"
     - Semantic Analysis: Identifies the query as related to cybersecurity, even though it doesn’t contain specific keywords like "security" or "threats."

2. **Match to Data Sources**:
   - The query’s meaning is compared to embeddings of available data sources or categories.
   - Example:
     - A "cybersecurity" embedding in the knowledge base closely matches the query’s semantic meaning.

3. **Route the Query**:
   - The system directs the query to the most relevant data source or module based on the semantic match.

### Simple Example
#### Query:
*"What are good practices for protecting APIs?"*

#### Semantic Routing:
1. **Semantic Understanding**:
   - The system analyzes the query and identifies it as related to "API security."
2. **Match to Data Sources**:
   - Compares the query's meaning to embeddings of knowledge bases like "API development" or "cybersecurity."
   - Finds that "cybersecurity" is the best match.
3. **Route the Query**:
   - Sends it to a cybersecurity knowledge base, even though the query doesn’t contain keywords like "threats" or "attacks."

### Why Semantic Routing Works
- **Flexibility**: It adapts to different ways users phrase their queries.
- **Context Awareness**: Captures the deeper intent and meaning of a query, not just surface keywords.
- **Robustness**: Handles ambiguous or unconventional phrasing more effectively than logical routing.

### Key Features of Semantic Routing
- **Embeddings**: Uses vector representations of queries and knowledge categories for similarity matching.
- **Intent Analysis**: Extracts the purpose of the query rather than relying on specific words.
- **Context Sensitivity**: Understands nuanced meanings, even when queries lack explicit details.

### Example Use Case
In an enterprise search system:
- Query: "How do I secure cloud data?"
- Semantic Match: Recognizes this as related to "cloud security," even though it doesn’t explicitly mention terms like "encryption" or "firewalls."
- Routing: Directs the query to a cloud security database.

### Benefits of Semantic Routing
- **Handles Complexity**: Routes ambiguous or context-dependent queries accurately.
- **Supports Natural Language**: Works well with conversational or informal phrasing.
- **Domain Independence**: Adapts to new domains without requiring extensive manual rule creation.

In short, **Semantic Routing** is like reading between the lines of a query to understand its true intent, then guiding it to the right destination based on meaning rather than just matching words. It’s flexible, intelligent, and ideal for handling diverse user queries!

## Setup

In [1]:
%run "../Z - Common/setup.ipynb"

Stored 'enable_langsmith' (bool)


USER_AGENT environment variable not set, consider setting it to identify your requests.


Semantic routing involves using embeddings to route a query to the most relevant prompt based on semantic similarity.

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.utils.math import cosine_similarity
from langchain.prompts import PromptTemplate

# Two prompts
template_physics = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

template_math = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

# Embed prompts
templates = [template_physics, template_math]
prompt_embeddings = embeddings.embed_documents(templates)

# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = templates[similarity.argmax()]
    # Chosen prompt 
    print("Using MATH" if most_similar == template_math else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)

In [3]:
chain.invoke("What's a black hole")

Using PHYSICS


"A black hole is an extremely dense region in space where gravity is so strong that nothing, not even light, can escape from it once it passes a boundary called the event horizon.\n\nTo break it down simply:\n1. It's created when massive stars die and collapse inward\n2. Its gravitational pull is so intense that once anything crosses the event horizon, it cannot escape\n3. We can't see black holes directly (since they don't emit light), but we can detect them by observing their effects on nearby matter\n4. They can range in size from stellar-mass black holes (a few times the mass of our Sun) to supermassive black holes (millions or billions of times the mass of our Sun) that exist at the centers of most galaxies\n\nThink of it like a cosmic drain - anything that gets too close gets pulled in, and once it passes the point of no return (event horizon), it can't get back out."

In [4]:
chain.invoke("What is 10 to the power of 5?")

Using MATH


"Let me solve this step by step.\n\n1) 10 to the power of 5 (written as 10⁵) means multiplying 10 by itself 5 times.\n\n2) Let's break it down:\n   * 10 × 10 = 100 (first two multiplications)\n   * 100 × 10 = 1,000 (third multiplication)\n   * 1,000 × 10 = 10,000 (fourth multiplication)\n   * 10,000 × 10 = 100,000 (fifth multiplication)\n\nTherefore, 10⁵ = 100,000\n\nAnother way to think about this is that 10⁵ means adding 5 zeros after 1."